In [1]:
import pandas as pd
import numpy as np
import requests
import time
import datetime
import json
import os

In [2]:
## File Path directory for writes
## data_store = os.mkdir(f'../data/defillama/ingest_files_{pd.Timestamp.now()}')

# This notebook ingests data from DeFiLlama and CoinGecko

Ideally there should be 4 tables

```
1. Table for the data raw data from DefiLlama: 
   - Function:raw_defillama()  
     - Table name: raw_defillama_df
2. Table for the cleaned and processed DefiLlama Data
   - Function: clean_defillama()
      - Table name: clean_defillama_df
2. Table for Total TVL
   - Function: Total_tvl
      - Table name: total_tvl_df
3. Table for protocol specific data
   - Function: protocol_data
      - Table name: protocol_data_df
4. Table for protocol chain specific related data
   - Function: protocol_chain_specific_data
      - Table name: protocol_chain_specific_data_df
5. Table for aggregated chain/native_chain/category data
   - Function: native_chain_agg
      - Table name: native_chain_agg_df
   - Function: category_agg
      - Table name: category_agg_df
   - Function: chain_specific_agg   
      - Table name: chain_specific_agg_df
```

# Function Store

## Helper Functions
You are likely to find these functions in other functions as opposed to being a standalone functions

In [3]:
def json_df_defillama(defillama_link):
    response = requests.get(defillama_link)
    df = pd.DataFrame(json.loads(json.dumps(response.json())))
    return df

def top_per_native_chain_category(protocol_list,mode=None, native_tvl=False,mcap=False,mcap_nativetvl=False):

    list_of_columns = ['id', 
                       'name', 
                       'symbol', 
                       'native_chain', 
                       'category',
                       'mcap',
                       'native_tvl',
                       'gecko_id', 
                       'cmcId',
                       'mcap_nativetvl',
                       'ingestion_time']
    
    if native_tvl+mcap+mcap_nativetvl>1:
        print('One must be true')
    elif  native_tvl+mcap+mcap_nativetvl==0:
        print('All cannot be false')
    elif mode==None:
        print('Please Choose a mode')

    elif mcap==True:
        #sorting by both columns
        df = protocol_list.loc[:,list_of_columns]
        df = df.sort_values('mcap',ascending=False)

        #create counter column used for later columns names
        df['rank'] = (df
                      .sort_values('mcap',ascending=False)
                      .groupby([f'{mode}'],as_index=False)
                      [['symbol','mcap']]
                      .cumcount()
                      .add(1))

        # Rank and order
        df = df.sort_values([f'{mode}','mcap','rank'],ascending=False)
        
        # Remove unneccessary columns
        df = df.loc[:,['id','rank','ingestion_time']]

        return df

    elif native_tvl==True:
        df = protocol_list.loc[:,list_of_columns]
        df = df.sort_values('native_tvl',ascending=False)

        #create counter column used for later columns names
        df['rank'] = (df
                      .sort_values('native_tvl',ascending=False)
                      .groupby([f'{mode}'],as_index=False)
                      [['symbol','native_tvl']]
                      .cumcount()
                      .add(1))

        # Rank and order
        df = df.sort_values([f'{mode}','native_tvl','rank'],ascending=False)
        
        # Remove unneccessary columns
        df = df.loc[:,['id','rank','ingestion_time']]

        return df
    
    # Smaller mcap/tvl is better
    elif mcap_nativetvl==True:
        df = protocol_list.loc[:,list_of_columns]
        df = df.sort_values('mcap_nativetvl',ascending=True)

        #create counter column used for later columns names
        df['rank'] = (df
                      .sort_values('mcap_nativetvl',ascending=True)
                      .groupby([f'{mode}'],as_index=False)
                      [['symbol','mcap_nativetvl']]
                      .cumcount()
                      .add(1))

        # Rank and order
        df = df.sort_values([f'{mode}','mcap_nativetvl','rank'],ascending=True)

        # Remove unneccessary columns
        df = df.loc[:,['id','rank','ingestion_time']]

        return df
    
def top_per_chain(protocol_chain_tvl):
    
    #sorting by both columns
    df = protocol_chain_tvl.sort_values('tvl_chain_specific',ascending=False)

    #create counter column used for later columns names
    df['rank'] = (df
                  .sort_values('tvl_chain_specific',ascending=False)
                  .groupby(['chains'],as_index=False)
                  [['symbol','tvl_chain_specific']]
                  .cumcount()
                  .add(1))

    # Rank and order
    df = df.sort_values(['chains','tvl_chain_specific','rank'],ascending=False)

    return df

## Table generating functions

In [4]:
def raw_defillama():
    """
    Desc:
        Ingests the data from DefiLlama Protocol endpoint
    Returns:
        A DataFrame
    
    """
    df = json_df_defillama('https://api.llama.fi/protocols')
    df['ingestion_time'] = pd.Timestamp.now() 
    return df

def clean_defillama(raw_defillama_df):
    """
    Desc:
        Removes tokens with 0 market cap, 
        get relavent columns, 
        computes mcap/tvl ratio,
        rename chain to native_chain
        rename tvl to native_tvl to avoid confusion
    Args:
        A list or array of columns to be sliced
    Returns:
        Cleaned dataFrame
    """
    # Get ingestion time
    columns_list = ['id', 
                'name', 
                'symbol', 
                'chain', 
                'audits', 
                'audit_note', 
                'gecko_id', 
                'cmcId', 
                'category', 
                'chains', 
                'slug', 
                'tvl', 
                'chainTvls',
                'change_1d', 
                'change_7d', 
                'staking', 
                'fdv', 
                'mcap',
                'mcap_nativetvl',
                'ingestion_time']
        
    raw_defillama_df['mcap_nativetvl'] = raw_defillama_df['mcap'] / raw_defillama_df['tvl']
    raw_defillama_df = raw_defillama_df.loc[:,columns_list]
    raw_defillama_df = raw_defillama_df[raw_defillama_df['mcap']>0]    
    raw_defillama_df['mcap_nativetvl'] = raw_defillama_df['mcap_nativetvl'].replace(np.inf, np.nan)
    raw_defillama_df['audits'] = raw_defillama_df['audits'].fillna(value=np.nan)
    raw_defillama_df['audits'] = raw_defillama_df['audits'].replace(np.nan, 0)
    cleaned_df = raw_defillama_df.rename({'chain':'native_chain','tvl':'native_tvl'},axis=1)
    
    return cleaned_df



def total_tvl():
    df = json_df_defillama('https://api.llama.fi/charts')
    df['date'] = pd.to_datetime(df['date'],unit='s')
    return df
    
########################################################################################################

def protocol_data(clean_defillama_df):
    """
    Desc:
        Generates a denormalized table with protocol specific data only and their ranks
    Args:
        Input the cleaned defillama df
    Returns:
        protocol_data and the corresponding ranks
    """
    protocol_df = clean_defillama_df.loc[:,['id', 
                                            'name', 
                                            'symbol', 
                                            'native_chain', 
                                            'audits', 
                                            'audit_note',
                                            'gecko_id', 
                                            'cmcId', 
                                            'category', 
                                            'native_tvl',
                                            'mcap',
                                            'mcap_nativetvl', 
                                            'ingestion_time']]
    
    # Permutations
    # Mode, native_tvl, mcap, mcap_nativetvl, table_name
    rank_permutations = [('native_chain',False,True,False,'rank_mcap_native_chain'),
                         ('native_chain',True,False,False,'rank_tvl_native_chain'),
                         ('category',False,True,False,'rank_mcap_cat'),
                         ('category',True,False,False,'rank_tvl_cat'),
                         ('native_chain',False,False,True,'rank_mcap_tvl_native_chain'),
                         ('category',False,False,True,'rank_mcap_tvl_cat')]
    
    for permutation in rank_permutations:
        df = top_per_native_chain_category(protocol_df,
                                           mode=permutation[0],
                                           native_tvl=permutation[1],
                                           mcap=permutation[2],
                                           mcap_nativetvl=permutation[3])
        
        
        protocol_df = (protocol_df.merge(df,
                                         how='left',
                                         on=['id','ingestion_time'])
                                  .rename({'rank':f'{permutation[4]}'},axis=1))                                       
    return protocol_df

def protocol_chain_specific_data(clean_defillama_df):
    chain_specific_df = clean_defillama_df.loc[:,['id',
                                                  'name',
                                                  'symbol',
                                                  'gecko_id',
                                                  'chains',
                                                  'chainTvls',
                                                  'ingestion_time']]
    
    chain_specific_df = chain_specific_df.explode('chains')
    tvl_chain = chain_specific_df.loc[:,['id','name','chainTvls']]
    
    tvl_chain = (pd.concat([tvl_chain, 
                            tvl_chain['chainTvls']
                            .apply(pd.Series)], axis=1)
                            .drop('chainTvls', axis=1))
    

    
    tvl_chain = pd.melt(tvl_chain, 
                             id_vars=['id','name'],
                             value_vars=tvl_chain.drop(['id','name'],axis=1).columns,
                             var_name='chains',
                             value_name='tvl')

    tvl_chain = tvl_chain.dropna().drop_duplicates()
    tvl_chain = tvl_chain.rename({'tvl':'tvl_chain_specific'},axis=1)
    
    chain_specific_df = (chain_specific_df.drop('chainTvls',axis=1)
                                          .merge(tvl_chain, 
                                                 how='left',
                                                 on=['id','name','chains']))
    
    chain_specific_df = top_per_chain(chain_specific_df)
    return chain_specific_df
    
def native_chain_agg(protocol_data_df):
    """
    Desc
        Aggregates the total market cap and tvl of a native chain
    Args
        Takes in the df generated by protocol data df
    Returns
        Returns the list of top native chains by market cap and tvl
        Ranked by market cap
    """
    list_of_columns = ['id','native_chain','mcap','native_tvl','ingestion_time']
    agg = (protocol_data_df.loc[:,list_of_columns]
                           .drop_duplicates()
                           .groupby(['native_chain','ingestion_time'],as_index=False)
                           [['mcap','native_tvl']].sum()
                           .sort_values('mcap',ascending=False))
    return agg

def category_agg(protocol_data_df):
    """
    Desc
        Aggregates the total market cap and tvl of a category
    Args
        Takes in the df generated by protocol data df
    Returns
        Returns the list of top native chains by market cap and tvl
        Ranked by market cap
    """
    list_of_columns = ['id','category','mcap','native_tvl','ingestion_time']
    agg = (protocol_data_df.loc[:,list_of_columns]
                           .drop_duplicates()
                           .groupby(['category','ingestion_time'],as_index=False)
                           [['mcap','native_tvl']].sum()
                           .sort_values('mcap',ascending=False))
    return agg

def chain_specific_agg(protocol_chain_specific_data_df):
    """
    Desc
        Aggregates the total market cap and tvl of a chain
    Args
        Takes in the df generated by protocol_chain_specific_data
    Returns
        Returns the list of top chain by market cap and tvl
        Ranked by market cap
    """
    list_of_columns = ['id','chains','tvl_chain_specific','ingestion_time']
    agg = (protocol_chain_specific_data_df.loc[:,list_of_columns]
                                          .drop_duplicates()
                                          .groupby(['chains','ingestion_time'],as_index=False)
                                          [['tvl_chain_specific']].sum()
                                          .sort_values('tvl_chain_specific',ascending=False))

    return agg
    

## Get raw defillama df
Function: raw_defillama()

In [5]:
raw_defillama_df = raw_defillama()

In [6]:
raw_defillama_df.head(2)

,id,name,address,symbol,url,description,chain,logo,audits,audit_note,...,staking,fdv,mcap,pool2,forkedFrom,listedAt,openSource,audit,audits_link,ingestion_time
0,3,Curve,0xD533a949740bb3306d119CC777fa900bA034cd52,CRV,https://curve.fi,Curve is a decentralized exchange liquidity po...,Multi-Chain,https://icons.llama.fi/curve.png,2,None,...,1.015280e+09,7.565838e+09,8.978093e+08,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-22 17:17:09.168395
1,118,MakerDAO,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,MKR,https://makerdao.com/,"Builders of Dai, a digital currency that can b...",Ethereum,https://icons.llama.fi/makerdao.jpg,2,None,...,NaN,2.070265e+09,1.855604e+09,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-22 17:17:09.168395


## Get cleaned defiLlama df
Function: clean_defillama

In [7]:
clean_defillama_df = clean_defillama(raw_defillama_df)


In [8]:
clean_defillama_df.audits.fillna(value=np.nan, inplace=True)

In [9]:
clean_defillama_df['audits'] = clean_defillama_df['audits'].fillna(value=np.nan)

In [10]:
clean_defillama_df[clean_defillama_df['id']=='118']

,id,name,symbol,native_chain,audits,audit_note,gecko_id,cmcId,category,chains,slug,native_tvl,chainTvls,change_1d,change_7d,staking,fdv,mcap,mcap_nativetvl,ingestion_time
1,118,MakerDAO,MKR,Ethereum,2,None,maker,1518,CDP,[Ethereum],makerdao,1.659372e+10,"{""Ethereum"": 16593718082.108505}",2.455863,11.609881,NaN,2.070265e+09,1.855604e+09,0.111826,2022-03-22 17:17:09.168395


In [11]:
clean_defillama_df.head(60)

,id,name,symbol,native_chain,audits,audit_note,gecko_id,cmcId,category,chains,slug,native_tvl,chainTvls,change_1d,change_7d,staking,fdv,mcap,mcap_nativetvl,ingestion_time
0,3,Curve,CRV,Multi-Chain,2,None,curve-dao-token,6538,Dexes,"[Ethereum, Avalanche, Fantom, Polygon, Arbitru...",curve,2.048584e+10,"{""Optimism"": 385934.71049077227, ""Avalanche"": ...",11.791334,16.105296,1.015280e+09,7.565838e+09,8.978093e+08,0.043826,2022-03-22 17:17:09.168395
1,118,MakerDAO,MKR,Ethereum,2,None,maker,1518,CDP,[Ethereum],makerdao,1.659372e+10,"{""Ethereum"": 16593718082.108505}",2.455863,11.609881,NaN,2.070265e+09,1.855604e+09,0.111826,2022-03-22 17:17:09.168395
2,182,Lido,LDO,Multi-Chain,2,None,lido-dao,8000,Liquid Staking,"[Ethereum, Terra, Solana, Moonriver]",lido,1.650219e+10,"{""Ethereum"": 8279239682.083258, ""Moonriver"": 3...",1.796294,15.956004,NaN,3.532524e+09,4.037554e+08,0.024467,2022-03-22 17:17:09.168395
3,294,Anchor,ANC,Multi-Chain,2,None,anchor-protocol,8857,Lending,"[Terra, Avalanche]",anchor,1.427032e+10,"{""Avalanche"": 47696890.308863945, ""Terra-borro...",0.412621,9.658175,NaN,2.698600e+09,7.185944e+08,0.050356,2022-03-22 17:17:09.168395
4,111,AAVE,AAVE,Multi-Chain,2,None,aave,7278,Lending,"[Ethereum, Avalanche, Polygon, Arbitrum, Fanto...",aave,1.316185e+10,"{""Optimism"": 725738.3241176717, ""Avalanche"": 3...",2.523760,16.912123,4.488759e+08,2.551963e+09,2.177615e+09,0.165449,2022-03-22 17:17:09.168395
5,319,Convex Finance,CVX,Ethereum,2,None,convex-finance,9903,Yield,[Ethereum],convex-finance,1.275837e+10,"{""Ethereum"": 12758365135.810318, ""Ethereum-sta...",1.387343,3.570451,7.670111e+08,2.063336e+09,1.128690e+09,0.088467,2022-03-22 17:17:09.168395
6,2,WBTC,WBTC,Ethereum,2,None,wrapped-bitcoin,3717,Bridge,[Ethereum],wbtc,1.163278e+10,"{""Ethereum"": 11632781010.240002}",3.333910,10.405338,NaN,1.160145e+10,1.160145e+10,0.997306,2022-03-22 17:17:09.168395
7,240,Polygon Bridge & Staking,MATIC,Polygon,0,None,matic-network,3890,Chain,[Polygon],polygon-bridge-&-staking,1.034399e+10,"{""Polygon"": 10343988565.534023}",-1.843771,1.418358,NaN,1.519120e+10,1.044074e+10,1.009354,2022-03-22 17:17:09.168395
8,1,Uniswap,UNI,Multi-Chain,2,None,uniswap,7083,Dexes,"[Ethereum, Polygon, Arbitrum, Optimism]",uniswap,7.554211e+09,"{""Optimism"": 40288911.046271086, ""Ethereum"": 7...",4.413567,NaN,NaN,9.982123e+09,4.556735e+09,0.603205,2022-03-22 17:17:09.168395
9,114,Compound,COMP,Ethereum,2,None,compound-governance-token,5692,Lending,[Ethereum],compound,6.990862e+09,"{""Ethereum"": 6990862469.905574, ""Ethereum-borr...",2.538552,9.243261,NaN,1.170809e+09,7.782137e+08,0.111319,2022-03-22 17:17:09.168395


# Get total TVL

In [12]:
total_tvl_df = total_tvl()

In [13]:
total_tvl_df.tail(1)

,date,totalLiquidityUSD
1235,2022-03-22,2.156773e+11


## Get Protocol Data
Function: protocol_data()

In [14]:
protocol_data_df = protocol_data(clean_defillama_df)

In [15]:
protocol_data_df.head()

,id,name,symbol,native_chain,audits,audit_note,gecko_id,cmcId,category,native_tvl,mcap,mcap_nativetvl,ingestion_time,rank_mcap_native_chain,rank_tvl_native_chain,rank_mcap_cat,rank_tvl_cat,rank_mcap_tvl_native_chain,rank_mcap_tvl_cat
0,3,Curve,CRV,Multi-Chain,2,None,curve-dao-token,6538,Dexes,2.048584e+10,8.978093e+08,0.043826,2022-03-22 17:17:09.168395,6,1,8,1,12,3
1,118,MakerDAO,MKR,Ethereum,2,None,maker,1518,CDP,1.659372e+10,1.855604e+09,0.111826,2022-03-22 17:17:09.168395,3,1,1,1,22,4
2,182,Lido,LDO,Multi-Chain,2,None,lido-dao,8000,Liquid Staking,1.650219e+10,4.037554e+08,0.024467,2022-03-22 17:17:09.168395,14,2,3,1,7,3
3,294,Anchor,ANC,Multi-Chain,2,None,anchor-protocol,8857,Lending,1.427032e+10,7.185944e+08,0.050356,2022-03-22 17:17:09.168395,8,3,4,1,17,9
4,111,AAVE,AAVE,Multi-Chain,2,None,aave,7278,Lending,1.316185e+10,2.177615e+09,0.165449,2022-03-22 17:17:09.168395,3,4,2,2,46,27


## Get Protocol chain specific data

In [16]:
protocol_chain_specific_data_df = protocol_chain_specific_data(clean_defillama_df)

In [17]:
protocol_chain_specific_data_df.head()

,id,name,symbol,gecko_id,chains,ingestion_time,tvl_chain_specific,rank
1075,1554,Kleros,PNK,kleros,xDai,2022-03-22 17:17:09.168395,NaN,16
980,696,Agave,AGVE,agave-token,xDai,2022-03-22 17:17:09.168395,NaN,15
937,299,Levinswap,LEVIN,levin,xDai,2022-03-22 17:17:09.168395,NaN,14
918,969,DeHive,DHV,dehive,xDai,2022-03-22 17:17:09.168395,NaN,13
877,838,Component,CMP,component,xDai,2022-03-22 17:17:09.168395,NaN,12


## Get Native chain agg rank
- Function: native_chain_agg

In [18]:
native_chain_agg_df = native_chain_agg(protocol_data_df)

In [19]:
native_chain_agg_df.head()

,native_chain,ingestion_time,mcap,native_tvl
12,Ethereum,2022-03-22 17:17:09.168395,4.092936e+10,6.950798e+10
22,Multi-Chain,2022-03-22 17:17:09.168395,2.438626e+10,1.015374e+11
29,Polygon,2022-03-22 17:17:09.168395,1.112548e+10,1.125301e+10
5,Binance,2022-03-22 17:17:09.168395,3.416888e+09,8.788286e+09
28,Osmosis,2022-03-22 17:17:09.168395,2.896032e+09,1.671348e+09


## Get category agg rank
- Function: category_agg

In [17]:
category_agg_df = category_agg(protocol_data_df)

/var/folders/hx/llk908mx5h77xg2bvkfqbhgm0000gn/T/ipykernel_58576/2497946747.py:171: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  agg = (protocol_data_df.loc[:,['id','category','mcap','native_tvl','ingestion_time']]


In [18]:
category_agg_df.head()

,category,ingestion_time,mcap,native_tvl
6,Dexes,2022-03-22 16:03:48.852909,3.511422e+10,5.947395e+10
1,Bridge,2022-03-22 16:03:48.852909,1.361860e+10,1.478842e+10
3,Chain,2022-03-22 16:03:48.852909,1.260853e+10,1.173626e+10
12,Lending,2022-03-22 16:03:48.852909,8.204406e+09,4.642614e+10
8,Gaming,2022-03-22 16:03:48.852909,4.095985e+09,1.844929e+07


## Get Chain specific agg
- Function: chain_specific_agg

In [19]:
chain_specific_agg_df = chain_specific_agg(protocol_chain_specific_data_df).head()

In [20]:
chain_specific_agg_df.head()

,chains,ingestion_time,tvl_chain_specific
19,Ethereum,2022-03-22 16:03:48.852909,1.298423e+11
53,Terra,2022-03-22 16:03:48.852909,2.530518e+10
44,Polygon,2022-03-22 16:03:48.852909,1.412980e+10
5,Binance,2022-03-22 16:03:48.852909,1.162376e+10
4,Avalanche,2022-03-22 16:03:48.852909,1.082211e+10


# Retrieving Token Historical Data with CoinGecko

We can build a portfolio using the segmentation model we found above. You can segment with a few ways:

Segment by mcap,tvl, or mcap_tvl size for native_chain or category.

Or you can segment tvl on the specific chain they are listed on.

Let's build a function that allows us to toggle

CoinGecko's API limit is fairly low at 50 calls per minute. To allow for buffers, we allow for 45 tokens to be listed for data retrieval. Hence would also need to check for such cases

1) Segment first

2) Check for 45 tokens or less

3) Pull data from coingecko

In [ ]:
master_table.columns

In [ ]:
def topn_segment(master_table, 
                 mode=None, 
                 metric=None, 
                 order=False, 
                 topn=10, 
                 rank_metric=None,
                 segment_filter=True,
                 topn_protocol=3):
    """
    Finds the top segment
    
    Args:
        Master_table as input
        Mode: cateogry, chain, or native chain
        Metric: category_agg_mcap/tvl, chain_specific_agg_mcap/tvl, native_chain_agg_mcap/tvl
        Order: True for asc, False for desc
        topn: TopN segments
        rank_metric: metric to rank and retain the relevant tokens in the segment
            Choose:
                'rank_mcap_native_chain', 'rank_tvl_native_chain',
                'rank_mcap_cat', 'rank_tvl_cat', 
                'rank_mcap_tvl_native_chain', 'rank_mcap_tvl_cat', 
                'rank_tvl_chain_specific'
        topn_protocol: Top n tokens in a protocol
        
    Returns: 
        A dataframe that is filtered and a dataframe that is segmented
    """
    topn_segment = (master_table
                    .loc[:,[mode, metric]]
                    .drop_duplicates()
                    .sort_values(metric, ascending=order)
                    .head(topn)
                    .drop(metric,axis=1)
                    .loc[:,mode].values)
    
    df = master_table[master_table.loc[:,mode].isin(topn_segment)]
    
    if mode != 'chain':
        df = df.loc[:,['id', 
                        'name', 
                        'symbol', 
                        'native_chain', 
                        'audits', 
                        'audit_note',
                        'gecko_id', 
                        'cmcId', 
                        'mcap', 
                        'native_tvl', 
                        'mcap_nativetvl', 
                        'category',
                        'ingestion_time', 
                        rank_metric]].drop_duplicates()
    else:
        df = df.loc[:,['id', 
                       'name', 
                       'symbol', 
                       'native_chain', 
                       'audits', 
                       'audit_note',
                       'gecko_id', 
                       'cmcId', 
                       'mcap', 
                       'native_tvl', 
                       'mcap_nativetvl', 
                       'category',
                       'ingestion_time', 
                       'chain', 
                       rank_metric]].drop_duplicates()
        
    if segment_filter is True:
        segment_filter = df[df[rank_metric]<=topn_protocol].sort_values(mode,ascending=False)
    else:
        segment_filter = None
        
    return df, segment_filter

In [ ]:
#     Args:
#         Master_table as input
#         Mode: cateogry, chain, or native chain
#         Metric: category_agg_mcap/tvl, chain_specific_agg_mcap/tvl, native_chain_agg_mcap/tvl
#         Order: True for asc, False for desc
#         topn: TopN segments
#         rank_metric: metric to rank and retain the relevant tokens in the segment
#             Choose:
#                 'rank_mcap_native_chain', 'rank_tvl_native_chain',
#                 'rank_mcap_cat', 'rank_tvl_cat', 
#                 'rank_mcap_tvl_native_chain', 'rank_mcap_tvl_cat', 
#                 'rank_tvl_chain_specific'
#         topn_protocol: Top n tokens in a protocol
#         topn_protocol_sort: True for asc, False for desc
        

In [ ]:
df,segment_filter = topn_segment(master_table_dedup, 
                                 mode='category', 
                                 metric='category_agg_tvl', 
                                 order=False, 
                                 topn=10, 
                                 rank_metric='rank_tvl_cat')

## Retrieving historical price for every protocol

In [ ]:
# Intentional code breaking mechanism to prevent api calls below
haha = hoohoo

In [ ]:
# Parameters for testing
start_date = '2021-03-01' 
start_unix = time.mktime(datetime.datetime.strptime(start_date, "%Y-%m-%d").timetuple())

end_date = '2022-03-12'
end_unix = time.mktime(datetime.datetime.strptime(end_date, "%Y-%m-%d").timetuple())

# coin_id = 'ethereum' # use the list in top_native_protocols_per_chain
vs_currency = 'usd'
from_date = start_unix
end_date = end_unix

In [ ]:
def json_df_coingecko(coin_id, vs_currency, start_unix, end_unix):
    link = f'https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range?vs_currency={vs_currency}&from={start_unix}&to={end_unix}'
    response = requests.get(link)
    df = pd.DataFrame(json.loads(json.dumps(response.json())))
        
    def extract_details(df_cell, position):
        return df_cell[position]

    df = df.drop(['market_caps','total_volumes'],axis=1)
    df['unix_timestamp'] = df['prices'].apply(lambda x: extract_details(x, 0))
    df['prices'] = df['prices'].apply(lambda x: extract_details(x, 1))
    df = df.rename({'prices':coin_id},axis=1)
    
    return df
    

In [ ]:
def get_historical_data(filtered_df, vs_currency,start_unix, end_unix):
    
    df = pd.DataFrame()
    ingested_count = 1
    processed_tokens = []
    culprit_coins = []

    ingestion_coinlist = filtered_df.gecko_id.values.tolist()
    # Force the ingestion of bitcoin and ethereum to make correlation or comparison analysis easier
    ingestion_coinlist.insert(0,'ethereum')
    ingestion_coinlist.insert(0,'bitcoin')
    
    for coin_id in ingestion_coinlist:
        while True:
            try:
                concat_df = json_df_coingecko(coin_id, vs_currency, start_unix, end_unix)
                print(f'Starting to process {coin_id}')
                
                if df.shape[1] == 0:
                    df = pd.concat([df, concat_df])
                else:
                    df = df.merge(concat_df, how='outer', on='unix_timestamp')
                
                print(f'{ingested_count} tokens ingested')
                ingested_count += 1
                processed_tokens.append(coin_id)
                print(f'Processed: {coin_id}')
                print(' ')
                break
                
                if ingested_count % 45 == 0:
                    # to prevent rate limits
                    time.sleep(65)
                    break
    
            except ValueError:
                print(f'Unknown Coin by coingecko')
                print('Removed processed tokens.')
                culprit_coins.append(coin_id)
                print(f'Culprit Coin {coin_id}')
                print(' ')    
                for coin in processed_tokens:
                    filtered_df = filtered_df[filtered_df.gecko_id != coin]      
                break
    
    # shift timestamp column in front so its neater
    col = df.pop("unix_timestamp")
    df.insert(0, col.name, col)
    
    # Convert the time format
    df['unix_timestamp'] = pd.to_datetime(df['unix_timestamp'],unit='ms')
    df = df.rename({'unix_timestamp':'date'},axis=1)
    df = df.set_index('date')
    
    return df, culprit_coins

In [ ]:
historical_prices, culprit_coin = get_historical_data(segment_filter, vs_currency, start_unix, end_unix)

In [ ]:
historical_prices

# File Generation Area
Where files are generated

In [ ]:
# To break the code on purpose
haha = hoohoo